In [1]:
!pip install rioxarray zarr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.1/284.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 105.0 MB/s eta 0:00:00


In [2]:
from collections.abc import Mapping

import gcsfs
from google.colab import auth
import folium
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import rioxarray
import shapely
import seaborn as sns
import numpy as np
import xarray as xr
import zarr

import warnings
# Suppress the FutureWarning
warnings.filterwarnings("ignore", category=FutureWarning)

In [4]:
project_id = 'ee-manmeet20singh15-wbis'

In [16]:
from google.cloud import storage

auth.authenticate_user()

fs = gcsfs.GCSFileSystem(project=project_id, token='cloud')
zstore = zarr.storage.FsspecStore(
    fs,
    path="weathernext/weathernext_2_0_0/zarr/2024_to_2025/predictions.zarr"
)

ds = xr.open_zarr(zstore)

# Show schema
ds

/tmp/ipython-input-1872479396.py:6: ZarrUserWarning: fs (<gcsfs.core.GCSFileSystem object at 0x7d773dc7dca0>) was not created with `asynchronous=True`, this may lead to surprising behavior
  zstore = zarr.storage.FsspecStore(


<xarray.Dataset> Size: 2PB
Dimensions:                   (time: 1464, sample: 64,
                               prediction_timedelta: 60, lat: 721, lon: 1440,
                               level: 13)
Coordinates:
  * time                      (time) datetime64[ns] 12kB 2024-01-01 ... 2024-...
  * sample                    (sample) int64 512B 0 1 2 3 4 5 ... 59 60 61 62 63
  * prediction_timedelta      (prediction_timedelta) timedelta64[ns] 480B 06:...
  * lat                       (lat) float32 3kB -90.0 -89.75 ... 89.75 90.0
  * lon                       (lon) float32 6kB 0.0 0.25 0.5 ... 359.5 359.8
  * level                     (level) int32 52B 50 100 150 200 ... 850 925 1000
    lead_time_secs            (prediction_timedelta) int64 480B dask.array<chunksize=(60,), meta=np.ndarray>
Data variables: (12/14)
    100m_u_component_of_wind  (time, sample, prediction_timedelta, lat, lon) float32 23TB dask.array<chunksize=(1, 4, 1, 721, 1440), meta=np.ndarray>
    100m_v_component_of_wind  (time, sample, prediction_timedelta, lat, lon) float32 23TB dask.array<chunksize=(1, 4, 1, 721, 1440), meta=np.ndarray>
    10m_u_component_of_wind   (time, sample, prediction_timedelta, lat, lon) float32 23TB dask.array<chunksize=(1, 4, 1, 721, 1440), meta=np.ndarray>
    10m_v_component_of_wind   (time, sample, prediction_timedelta, lat, lon) float32 23TB dask.array<chunksize=(1, 4, 1, 721, 1440), meta=np.ndarray>
    2m_temperature            (time, sample, prediction_timedelta, lat, lon) float32 23TB dask.array<chunksize=(1, 4, 1, 721, 1440), meta=np.ndarray>
    geopotential              (time, sample, prediction_timedelta, level, lat, lon) float32 304TB dask.array<chunksize=(1, 4, 1, 13, 721, 1440), meta=np.ndarray>
    ...                        ...
    specific_humidity         (time, sample, prediction_timedelta, level, lat, lon) float32 304TB dask.array<chunksize=(1, 4, 1, 13, 721, 1440), meta=np.ndarray>
    temperature               (time, sample, prediction_timedelta, level, lat, lon) float32 304TB dask.array<chunksize=(1, 4, 1, 13, 721, 1440), meta=np.ndarray>
    total_precipitation_6hr   (time, sample, prediction_timedelta, lat, lon) float32 23TB dask.array<chunksize=(1, 4, 1, 721, 1440), meta=np.ndarray>
    u_component_of_wind       (time, sample, prediction_timedelta, level, lat, lon) float32 304TB dask.array<chunksize=(1, 4, 1, 13, 721, 1440), meta=np.ndarray>
    v_component_of_wind       (time, sample, prediction_timedelta, level, lat, lon) float32 304TB dask.array<chunksize=(1, 4, 1, 13, 721, 1440), meta=np.ndarray>
    vertical_velocity         (time, sample, prediction_timedelta, level, lat, lon) float32 304TB dask.array<chunksize=(1, 4, 1, 13, 721, 1440), meta=np.ndarray>
Attributes:
    label:                    forecasts_15d
    model_xid:                193484016
    model_wid:                45
    evaluation_xid:           194846205
    evaluation_wid:           1
    model_input_data_type:    default
    use_synthetic_radiation:  False
    start_forecast_date:      2024-01-01
    end_forecast_date:        2025-01-01
    time_between_forecasts:   6 hours
    random_seed:              0

In [13]:
import gcsfs

fs = gcsfs.GCSFileSystem(
    project=project_id,
    token="cloud",
    asynchronous=False,   # <-- key
)
print(fs.ls("weathernext/weathernext_2_0_0/zarr"))

['weathernext/weathernext_2_0_0/zarr/2022_to_2023', 'weathernext/weathernext_2_0_0/zarr/2023_to_2024', 'weathernext/weathernext_2_0_0/zarr/2024_to_2025', 'weathernext/weathernext_2_0_0/zarr/2025_to_present', 'weathernext/weathernext_2_0_0/zarr/weathernext_2_0_0_file_structure.pdf']


In [15]:
import gcsfs

fs = gcsfs.GCSFileSystem(
    project=project_id,
    token="cloud",
    asynchronous=False,   # <-- key
)
print(fs.ls("weathernext/weathernext_2_0_0/zarr/2024_to_2025"))

['weathernext/weathernext_2_0_0/zarr/2024_to_2025/', 'weathernext/weathernext_2_0_0/zarr/2024_to_2025/predictions.zarr', 'weathernext/weathernext_2_0_0/zarr/2024_to_2025/success']


In [19]:
import gcsfs

fs = gcsfs.GCSFileSystem(
    project=project_id,
    token="cloud",
    asynchronous=False,   # <-- key
)
print(fs.ls("weathernext/59572747_4_0/zarr"))

['weathernext/59572747_4_0/zarr/', 'weathernext/59572747_4_0/zarr/132879806_2023_to_2024', 'weathernext/59572747_4_0/zarr/132880704_2022_to_2023', 'weathernext/59572747_4_0/zarr/59572747_4_2024_to_present', 'weathernext/59572747_4_0/zarr/99140631_1_2020_to_2021', 'weathernext/59572747_4_0/zarr/99140631_2_2021_to_2022', 'weathernext/59572747_4_0/zarr/weathernext_graph_file_structure.pdf']


In [20]:
import gcsfs

fs = gcsfs.GCSFileSystem(
    project=project_id,
    token="cloud",
    asynchronous=False,   # <-- key
)
print(fs.ls("weathernext/59572747_4_0/zarr/59572747_4_2024_to_present"))

['weathernext/59572747_4_0/zarr/59572747_4_2024_to_present/', 'weathernext/59572747_4_0/zarr/59572747_4_2024_to_present/20240101_00hr_01_preds', 'weathernext/59572747_4_0/zarr/59572747_4_2024_to_present/20240101_06hr_01_preds', 'weathernext/59572747_4_0/zarr/59572747_4_2024_to_present/20240101_12hr_01_preds', 'weathernext/59572747_4_0/zarr/59572747_4_2024_to_present/20240101_18hr_01_preds', 'weathernext/59572747_4_0/zarr/59572747_4_2024_to_present/20240102_00hr_01_preds', 'weathernext/59572747_4_0/zarr/59572747_4_2024_to_present/20240102_06hr_01_preds', 'weathernext/59572747_4_0/zarr/59572747_4_2024_to_present/20240102_12hr_01_preds', 'weathernext/59572747_4_0/zarr/59572747_4_2024_to_present/20240102_18hr_01_preds', 'weathernext/59572747_4_0/zarr/59572747_4_2024_to_present/20240103_00hr_01_preds', 'weathernext/59572747_4_0/zarr/59572747_4_2024_to_present/20240103_06hr_01_preds', 'weathernext/59572747_4_0/zarr/59572747_4_2024_to_present/20240103_12hr_01_preds', 'weathernext/59572747_4_0